## 🔁 GRU(Gated Recurrent Unit) 구조와 동작 원리

### ✅ LSTM의 장점을 가져온 GRU

GRU(Gated Recurrent Unit)는 **LSTM의 장점을 유지하면서 더 단순한 구조**로 만든 순환 신경망(RNN) <br>
LSTM의 핵심 기능인 **장기 기억 유지**는 그대로 가져오되, 내부 구조를 더 간소화해서 **연산량을 줄이고 학습 속도를 개선**


### 🔍 GRU의 핵심 특징

- LSTM에서 사용되던 3개의 게이트 (Forget, Input, Output)를  
  → 2개의 게이트 (Update, Reset)로 축소
- LSTM의 **Cell State** $C_t$를 제거하고,  
  → 하나의 **Hidden State** $h_t$가 모든 기억을 담당
- 구조는 단순하지만, 성능은 LSTM에 근접하거나 유사


### 📦 GRU의 구조 개요

- GRU는 은닉 상태 $h_{t-1}$에서 다음 상태 $h_t$로 연결될 때  
  **두 갈래로 분기되었다가 다시 합쳐지는 구조(장기기억과 단기기억)** 를 가짐! (LSTM이랑 비슷)

![gru_cell](figures/rnn/23_gru_cell.png)    

###  🧠 GRU (Gated Recurrent Unit) 계산 흐름
##### ✅ Step 0. 입력 정보
- $x_t$: 현재 시점의 입력 벡터 (예: 단어 임베딩)
- $h_{t-1}$: 이전 시점의 은닉 상태 (과거 정보의 요약)
#### 🧩 Step 1. Reset Gate 계산
**→ 과거 정보를 얼마나 '초기화'할지 결정**

$r_t = \sigma(W_r x_t + U_r h_{t-1})$
- $\sigma$: sigmoid 함수 (출력값 0~1)
- $W_r$: 입력 $x_t$에 대한 가중치
- $U_r$: 이전 상태 $h_{t-1}$에 대한 가중치
- 👉 $r_t$가 작을수록 과거 정보를 무시하고 새 정보를 더 반영

🧠 **의도**: 문맥이 바뀔 때 이전 기억을 지워야 할 경우 사용됨

#### 🧩 Step 2. Update Gate 계산
**→ 현재 상태에 과거를 얼마나 유지할지 결정**

$z_t = \sigma(W_z x_t + U_z h_{t-1})$
- $z_t \approx 1$: 이전 상태 $h_{t-1}$를 거의 그대로 유지  
- $z_t \approx 0$: 새로 계산된 정보를 더 반영

🧠 **의도**: 장기 기억이 필요한 경우, 과거 상태를 그대로 유지

#### 🧪 Step 3. Candidate Hidden State 계산
**→ 현재 입력과 선택된 과거 정보로 새로운 상태 후보 생성**

$\tilde{h}_t = \tanh(W x_t + U (r_t \cdot h_{t-1}))$
- $r_t \cdot h_{t-1}$: reset gate로 조절된 과거 정보
- $\tanh$: 비선형 활성화 함수 (출력 범위 -1 ~ 1)

🧠 **의도**: 현재 입력과 일부 과거 정보를 이용해 새 정보를 생성

#### ⚙️ Step 4. 최종 Hidden State 계산
**→ 새 정보와 과거 정보를 비율에 따라 혼합**

$h_t = z_t \cdot h_{t-1} + (1 - z_t) \cdot \tilde{h}_t$
- $z_t$: 이전 상태를 유지할 정도
- $1 - z_t$: 새로운 상태 후보를 반영할 정도

🧠 **의도**: 필요에 따라 '기억 유지' 또는 '기억 갱신'을 자동으로 수행

 ### 📘Pytorch GRU
- `nn.GRU` 클래스 이용
    - https://pytorch.org/docs/stable/generated/torch.nn.GRU.html
- **입력**
    - **input**: (seq_length, batch, hidden_size) shape의 tensor. (batch_first=False), batch_first=True이면 `seq_length`와 `batch` 위치가 바뀐다.
    - **hidden**: (D * num_layers, batch, hidden_size) shape의 Tensor. D(양방향:2, 단방향:1), hidden은 생략하면 0이 입력됨.
- **출력** - output과 hidden state가 반환된다.
    - **output**
        - 모든 sequence의 처리결과들을 모아서 제공.
        - shape: (seq_length, batch, D * hidden_size) : D(양방향:2, 단방향:1), batch_first=True이면 `seq_length`와 `batch` 위치가 바뀐다.
    - **hidden**
        - 마지막 time step 처리결과
        - shape: (D * num_layers, batch, hidden) : D(양방향:2, 단방향:1)


📝 다음 코드의 빈칸을 채우고 주석을 참고하여 작성하세요.

In [1]:
## GRU 입출력  확인
import torch
import torch.nn as nn

# dummy data -> (seq len-20, batch-2, 개별 timestep의 입력 feature수-10으로 설정해주세요!)
input_data = torch.randn((20,2,10))

In [2]:
# 단방향, layer 수 : 1
gru1 = nn.GRU(  # GRU 계층을 입력하세요!
    input_size=10,   # 개별 timestep의 feature 수(embedding.dim)
    hidden_size=256, # 각 timestep별로 256개의 특성을 추출(unit)
    num_layers=1, # 몇층(layer)를 쌓을 지. (default: 1)
    bidirectional=False # 양방향 여부 (default: False)
)
out1, hidden1 = gru1(input_data)
#모든 timestep의 hidden state값을 묶어서 반환.[20:seq len, 2:batch, 256:hidden_size] 
# 256개의 값으로 구성되어 있는 20개의 seq 값.
print(out1.shape) 
# 마지막 timestep 처리 hidden state값 [1: seq len, 2, 256]
print(hidden1.shape)

torch.Size([20, 2, 256])
torch.Size([1, 2, 256])


In [3]:
# 양방향, layer 수: 1
gru2 = nn.GRU(  # GRU 계층을 입력하세요!
    input_size=10,
    hidden_size=256, 
    num_layers=1, # 몇층(layer)를 쌓을 지. (default: 1)
    bidirectional=True# 양방향 여부 (default: False)
)
out2, hidden2 = gru2(input_data)

# [20: seq_len, 2: batch, 512:hidden_size * 2]  양방향(정/역방향) hidden state를 합쳐서(concat) 반환.
print(out2.shape)
# [2:정/역방향 두개, 2:batch, 256:hidden size]
print(hidden2.shape)

torch.Size([20, 2, 512])
torch.Size([2, 2, 256])


# Seq2Seq 를 이용한 Chatbot 모델 구현
- Encoder를 이용해 질문의 특성을 추출하고 Decoder를 이용해 답변을 생성한다.

### 📘데이터 로드

In [ ]:
# requests 모듈로 받기
import requests
import os

os.makedirs("data", exist_ok=True)

In [ ]:
url = "https://raw.githubusercontent.com/songys/Chatbot_data/refs/heads/master/ChatbotData.csv"
res = requests.get(url)
if res.status_code == 200:
    with open("data/chatbot_data.csv", "wt", encoding="utf-8") as fw:
        fw.write(res.text)
else:
    print(f"불러오지 못함: {url}")

In [ ]:
import pandas as pd
df = pd.read_csv('data/chatbot_data.csv')
df.shape

(11823, 3)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11823 entries, 0 to 11822
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Q       11823 non-null  object
 1   A       11823 non-null  object
 2   label   11823 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 277.2+ KB


In [ ]:
# 라벨 제거
df.drop(columns='label', inplace=True)

In [ ]:
df.head()

,Q,A
0,12시 땡!,하루가 또 가네요.
1,1지망 학교 떨어졌어,위로해 드립니다.
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.
4,PPL 심하네,눈살이 찌푸려지죠.


In [ ]:
df.tail()

,Q,A
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.
11820,흑기사 해주는 짝남.,설렜겠어요.
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.
11822,힘들어서 결혼할까봐,도피성 결혼은 하지 않길 바라요.


In [ ]:
df.isna().sum()  # 결측치 확인

Q    0
A    0
dtype: int64

### 📘Dataset, DataLoader 정의

### 📘 Tokenization

### Subword방식

📝 다음 코드의 빈칸을 채우고 주석을 참고하여 작성하세요.

In [ ]:
# seq2seq에서는 정규화 같은 전처리가 필요하지 않다. 부정과 긍정을 판단하는 RSTM 같은 것들은 전처리나 클랜징을 필요로 함. 
# token 학습 -> vocab 사전 생성.
## 질문들 + 답변들 합쳐서 학습.
question_texts = df['Q']
answer_texts = df['A']
all_texts = list(question_texts + " "+answer_texts) # 같은 index끼리 합치기 => list로 변환
len(question_texts), len(answer_texts), len(all_texts)

In [ ]:
all_texts[:5]

In [ ]:
question_texts[:5]

In [ ]:
answer_texts[:5]

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

In [ ]:
vocab_size = 10_000   # 총 어휘 수
min_frequency = 5   # 어휘사전에 등록될 단어(토큰)의 최소 빈도수.

tokenizer = Tokenizer(BPE(unk_token="[UNK]"))  # unk 처리
tokenizer.pre_tokenizer = Whitespace()  # 음절 단위로 쪼갬.
trainer = BpeTrainer(      
    vocab_size=vocab_size,
    min_frequency=min_frequency,
    continuing_subword_prefix='##', # 연결 subword 앞에 붙일 접두어지정. 
    special_tokens=["[PAD]", "[UNK]", "[SOS]"] # [SOS]: 문장의 시작을 의미하는 토큰.
)
# tokenizer: token + ##izer
## 학습
tokenizer.train_from_iterator(all_texts, trainer=trainer) # 리스트로 부터 학습
## tokenizer.train("파일경로") # 파일에 있는 text를 학습.

In [ ]:
print("총 어휘수:", tokenizer.get_vocab_size())

In [ ]:
# 토큰화
encode = tokenizer.encode("오늘 날씨가 너무 좋습니다. 즐거운 하루 되세요. 쿄쿄쿜ㅋ")    # id와 토큰을 생성해줌. 
print(encode.ids)   # 토큰 id
print(encode.tokens)  # 토큰 (단어)

In [ ]:
tokenizer.id_to_token(1296), tokenizer.token_to_id('##삿')

### 📘 Tokenizer 저장

### 📘 Dataset, DataLoader 정의


### Dataset 정의 및 생성
- 모든 문장의 토큰 수는 동일하게 맞춰준다.
    - DataLoader는 batch 를 구성할 때 batch에 포함되는 데이터들의 shape이 같아야 한다. 그래야 하나의 batch로 묶을 수 있다.
    - 문장의 최대 길이를 정해주고 **최대 길이보다 짧은 문장은 `<PAD>` 토큰을 추가**하고 **최대길이보다 긴 문장은 최대 길이에 맞춰 짤라준다.**

📝 다음 코드의 빈칸을 채우고 주석을 참고하여 작성하세요.

In [ ]:
import random
import os
import numpy as np
import time

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler
from torch import optim

device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
class ChatbotDataset(Dataset):
    """
    ChatbotDataset
    parameter:
        question_texts: list[str] - 질문 texts 목록. 리스트에 질문들을 담아서 받는다. ["질문1", "질문2", ...]
        answer_texts: list[str] - 답 texts 목록. 리스트에 답변들을 담아서 받는다.     ["답1", "답2", ...]
        max_length: 개별 문장의 token 개수. 모든 문장의 토큰수를 max_length에 맞춘다.
        tokenizer: Tokenizer
        vocab_size: int 총단어수
    """

    def __init__(self, question_texts, answer_texts, max_length, tokenizer):
        """
        parameter
            question_texts: list[str] - 질문 texts 목록. 리스트에 질문들을 담아서 받는다. ["질문1", "질문2", ...]
            answer_texts: list[str] - 답 texts 목록. 리스트에 답변들을 담아서 받는다.     ["답1", "답2", ...]
            max_length: 개별 문장의 token 개수. 모든 문장의 토큰수를 max_length에 맞춘다.
            tokenizer: Tokenizer
        """
        self.max_length = max_length
        self.tokenizer = tokenizer
        self.question_texts = [self.__process_sequence(q) for q in question_texts]
        self.answer_texts = [self.__process_sequence(a) for a in answer_texts]  # 받아온 q/a 문장 리스트를 max_length의 token list로 변환해서 attribute로 저장.
    
    def __pad_token_sequence(self, token_sequence): 
        """
        max_length 길이에 맞춰 token_id 리스트를 구성한다.
        max_length 보다 길면 뒤에를 자르고 max_length 보다 짧으면 [PAD] 토큰을 추가한다.
        
        Parameter
            token_sentence: list[int] - 길이를 맞출 한 문장 token_id 목록
        Return
            list[int] - length가 max_length인 token_id 목록
        """
        pad_token = self.tokenizer.token_to_id('[PAD]')   # [PAD] 토큰 id를 조회 (0)
        seq_len = len(token_sequence) # 입력 문장의 토큰수
        if seq_len > self.max_length: # 문장 최대 토큰수 보다 길다면.
            return token_sequence[:self.max_length]
        else:
            return token_sequence + ([pad_token] * (self.max_length - seq_len))
    
    def __process_sequence(self, text): 
        """
        한 문장(str)을 받아서 padding이 추가된 token_id 리스트로 변환 후 반환
        Parameter
            text: str - token_id 리스트로 변환할 한 문장
        Return
            list[int] - 입력받은 문장에 대한 token_id 리스트
        """
        # encoding
        encode = self.tokenizer.encode(text) # "........" => [. , . , .]
        # max_length 크기에 맞춘다.
        token_ids = self.__pad_token_sequence(encode.ids) #[3400, 20, 6, 0, 0, 0 ..]
        return token_ids
    
    def __len__(self):
        return len(self.question_texts)

    
    def __getitem__(self, index):
        # return  index의 (질문토큰들,  답변토큰들)
        q = self.question_texts[index]  # List
        a = self.answer_texts[index]
        # List->LongTensor. nn.Embedding()의 입력(정수타입)으로 들어간다. 
        return torch.tensor(q, dtype=torch.int64), torch.tensor(a, dtype=torch.int64)

In [ ]:
# 적당한 max_length 값: 전체 문장 총 토큰수의 9분위수
import numpy as np
a = [len(tokenizer.encode(s).ids) for s in all_texts]
# a[:5]
np.quantile(a, q=[0.9, 0.95, 0.97, 1.0])
# max_length=20
a  # 각 문장에 대한 토큰 수

In [ ]:
import matplotlib.pyplot as plt
plt.hist(a, 30);

In [ ]:
############# Dataset 셍상
MAX_LENGTH = 20
dataset = ChatbotDataset(question_texts, answer_texts, MAX_LENGTH, tokenizer)
len(dataset)

In [ ]:
dataset[0]  
# (tensor([  10, 1815, 1348,  368,    3,    0,    0,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0]),  -> 질문 
# tensor([6118,  378,   47, 2252,    8,    0,    0,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0])) -> 답

### 📘 Trainset / Testset 나누기
train : test = .95 : .05

In [ ]:
int(len(dataset) * 0.95)
len(dataset) - int(len(dataset)*0.95)

In [ ]:
train_size = int(len(dataset)*0.95)   # trainset의 개수
test_size = len(dataset) - train_size  # testset의 개수
print(train_size, test_size)

In [ ]:
# random_split()이용해서 분리
###shuffle(섞는다) 후 개수에 맞게 나눔  -> 순서대로 나누지 않음.
train_set, test_set = random_split(dataset, [train_size, test_size]) 
# random_split(Dataset객체, [나눌 개수, ...])
# ex. random_split(dataset, [10,20,30,40,50]) # 5개로 나눔. 각각 지정한 개수별로.

In [ ]:
type(dataset), type(train_set)  # subset (부분 집합)의 형태로 나옴. 

In [ ]:
len(train_set), len(test_set)

### 📘 DataLoader 생성

In [ ]:
BATCH_SIZE = 64
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)  #drop_last= True 남은거는 쓰지 않겠다.
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE)

In [ ]:
len(train_loader), len(test_loader) # step 수

### 📘 모델 정의

## Seq2Seq 모델 정의
- Seq2Seq 모델은 Encoder와 Decoder의 입력 Sequence의 **길이**와 **순서**가 자유롭기 때문에 챗봇이나 번역에 이상적인 구조다.
    - 단일 RNN은 각 timestep 마다 입력과 출력이 있기 때문에 입/출력 sequence의 개수가 같아야 한다.(ex. many to many)
    - 챗봇의 질문/답변이나 번역의 대상/결과 문장의 경우는 사용하는 어절 수가 다른 경우가 많기 때문에 단일 RNN 모델은 좋은 성능을 내기 어렵다.
    - Seq2Seq는 **입력처리(질문,번역대상)처리 RNN과 출력 처리(답변, 번역결과) RNN 이 각각 만들고 그 둘을 연결한 형태로 길이가 다르더라도 상관없다.**

## 📘 Encoder
Encoder는 하나의 Vector를 생성하며 그 Vector는 **입력 문장의 의미**를 N 차원 공간 저장하고 있다. 이 Vector를 **Context Vector** 라고 한다.<br> 
LSTM과 동일(Many to one) -> LSTM에서 특징 추출해내는 거랑 똑같이 여기서도 입력 문장의 context의 특징을 추출해내는 것
![encoder](figures/seq2seq_encoder.png)

📝 다음 코드의 빈칸을 채우고 주석을 참고하여 작성하세요.

In [ ]:
class Encoder(nn.Module):

    def __init__(self, vocab_size, embedding_dim, 
                hidden_size, bidirectional=True, num_layers=1, dropout_rate=0.0):  #bidirectional : 양방향성.  문장 token에 대해서 왼쪽에서 뽑아낸거랑 오른른쪽에서 시작해서 뽑아낸거 둘다 고려.
        super().__init__()
        # Encoder는 context vector(문장의 feature)를 생성하는 것이 목적 (분류기는 생성안함.) 여기서는 그냥 특징을 decoder에 주는 역할
        # Embedding Layer, GRU Layer를 생성.
        self.vocab_size = vocab_size # 어휘사전의 총 어휘수(토큰수)
        # 임베딩레이어
        self.embedding = nn.Embedding(
            vocab_size,		# 총 어휘개수 (weight 행렬의 행)
            embedding_dim,	# embedding vector 차원수. (Weight 행렬의 열 수) weight 행렬의 shape: [vocab_size, embedding_dim]
            padding_idx=0   # [PAD]  (패딩 토큰의 ID) - padding의 embedding vector는 학습이 안되도록 한다.(vector값이 0으로 구성)
        )
        # GRU
        self.gru = nn.GRU(
            embedding_dim,				# 개별 토큰(time step)의 크기(feature 수).
            hidden_size=hidden_size,	# hidden state의 크기- 개별 토큰 별로 몇개의 feature를 추출할지. 
            num_layers=num_layers,
            bidirectional=bidirectional,
            dropout=dropout_rate if num_layers > 1 else 0.0  # stacked rnn일 경우(layer가 여러개일 경우), dropout 적용.  
        )
    
    def forward(self, X):   # 계산
        # X shape: (batch, seq_len) 토큰값 하나씩
        X = self.embedding(X) # (batch, seq_len, embedding_dim)
        X = X.transpose(1, 0) # (seq_len, batch, embedding_dim)
        out, hidden = self.gru(X)
        return out, hidden

In [ ]:
# padding_idx 예시
e_l = nn.Embedding(10,5, padding_idx=1)
e_l.weight

In [ ]:
from torchinfo import summary
encoder_model = Encoder(1000, 200, 256)   # vocab_size : 1000, emb차원: 200, hidden size: 256
dummy_data = torch.zeros((64, 20), dtype=torch.int64)  #(batch:64, seq_len:20)
summary(encoder_model, input_data=dummy_data)

## 📘Decoder
- Encoder의 출력(context vector)를 받아서 번역 결과 sequence를 출력한다.
- Decoder는 매 time step의 입력으로 **이전 time step에서 예상한 단어와 hidden state값이** 입력된다.
- Decoder의 처리결과 hidden state를 Estimator(Linear+Softmax)로 입력하여 **입력 단어에 대한 번역 단어가 출력된다.** (이 출력단어가 다음 step의 입력이 된다.)
    - Decoder의 첫 time step 입력은 문장의 시작을 의미하는 <SOS>(start of string) 토큰이고 hidden state는 context vector(encoder 마지막 hidden state) 이다.

![decoder](figures/seq2seq_decoder.png)

📝 다음 코드의 빈칸을 채우고 주석을 참고하여 작성하세요.

In [ ]:
class Decoder(nn.Module):
    # auto regressive RNN 모델은 단방향만 가능
    def __init__(self, vocab_size, embedding_dim, 
                 hidden_size, num_layers=1, bidirectional=False, dropout_rate=0.0):  #여기서는 역방향을 할 수 없음. 앞에서 생성이 되어야 하는데, 뒤로 하면 생성 안됨. 
        super().__init__()
        self.vocab_size = vocab_size # 총 어휘사전 토큰 개수.
        # embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        # GRU 
        ## Auto Regressive RNN은 단방향만 가능. 
        self.gru = nn.GRU(embedding_dim, hidden_size, 
                          num_layers=num_layers, dropout=dropout_rate if num_layers > 1 else 0.0)
        # Dropout layer (feature랑 분류기 사이에 dropout layer 넣기)
        self.dropout = nn.Dropout(dropout_rate)
        # 분류기 (다음 단어(토큰)를 추론)
           # - 다중분류(단어사전의 단어들의 다음 단어일 확를)
        self.lr = nn.Linear(
            hidden_size,  # GRU 출력 값 중 마지막 hidden state값을 입력으로 받음.  # ev -> --- -> hidden state -> linnear에 넣기
            vocab_size    # 출력: 다음 단어일 확률.   
        )
        
    def forward(self, X, hidden):
        # X: torch.LongTensor: shape - [batch] : 한 단어씩 입력을 받음.
        # hidden: torch.FloatTensor: shape - [1, batch, hidden_size] (이전까지의 특성)   # sequence_length는 1이 됨 (단어가 한개)
        
        X = X.unsqueeze(1) # seq_len 축을 추가. [batch] -> [batch, 1] (Embedding Layer의 input shape)  1: sequance_length
        X = self.embedding(X) # [batch, 1, embedding 차원]
        X = X.transpose(1, 0) # [1, batch, embedding 차원]  #seq_len이랑 batch 축 바꿈. 
        
        out, hidden = self.gru(X, hidden)
        last_out = out[-1] # out: 전체 hidden state값-> 마지막 hidden state을 추출  # 근데 어차피 seq_len이 1이니까 한개임.
        self.dropout(last_out)   # 과적합을 막아주기 위해서 dropout 진행
        last_out = self.lr(last_out)

        #last_out : 어휘 사전의 단어들에 대해 다음 단어일 확률. 
        return last_out, hidden # (hidden: 다음 timestep에 전달.)  # hidden도 같이 다음 꺼에 넣어야 함,

In [ ]:
#### summary

decoder_model = Decoder(1000, 200, 256)

dummy_input = torch.ones((64, ), dtype=torch.int64)
dummy_hidden = torch.ones((1, 64, 256), dtype=torch.float32)

summary(decoder_model, input_data=(dummy_input, dummy_hidden))